In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)

from trl import SFTTrainer

from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=False,
)
device_map = {"": 0}


In [ ]:


tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")
model = AutoModelForCausalLM.from_pretrained(
    "tiiuae/falcon-7b", quantization_config=bnb_config, device_map=device_map, trust_remote_code=True,
    load_in_8bit=True
)
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)
model_id = "/home/lars/ML/LLM/results/checkpoint-20"
model = PeftModel.from_pretrained(model, model_id)
model.config.use_cache = False


In [ ]:
# %%timeit -r 5
# inference test
with torch.inference_mode():
    text = '### Human: Write a short dialogue between two Norwegian people, Cristiana and Waleed, who just met.### Assitant:'
    outputs = model.generate(input_ids=tokenizer(text, return_tensors="pt").input_ids, 
                            max_new_tokens=100, 
                            temperature=0.8, 
                            num_return_sequences=1,
                            do_sample=False,
                            use_cache = True,
                            top_k=50)

print(tokenizer.batch_decode(outputs))


In [ ]:
## So inference time is probably significantly slower than the openai api, but interesting to experiment with further



In [ ]:
from datasets import load_dataset

dataset_name = "timdettmers/openassistant-guanaco"
dataset = load_dataset(dataset_name, split="train")

In [ ]:
import json
from datasets import load_dataset

# Read the JSON data
with open('big_data.json') as f:
    data = json.load(f)

# Transform the data into a list of dictionaries, each with a single key-value pair
transformed_data = [{"id": key, "text": value} for key, value in data.items()]

# Save the transformed data to a new JSON file
with open('transformed.json', 'w') as f:
    for item in transformed_data:
        json.dump(item, f)
        f.write('\n')

# Load the transformed data into a HuggingFace Dataset
dataset = load_dataset('json', data_files='transformed.json', split='train')

# Let's check the first element in the dataset
print(dataset[0])


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config=bnb_config, 
    trust_remote_code=True
)
model.config.use_cache = False

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
## Finetune

from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)



In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()